In [44]:
import time
import cv2 as cv
import numpy as np

import import_ipynb
import Person

In [45]:
counter = 0
cap = cv.VideoCapture('TestVideo.avi')
#cap = cv.VideoCapture('test2.mp4')

w = 640
h = 480
line_down = 350
down_limit = 410

line_down_color = (255,0,0)
pt1 =  [0, line_down];
pt2 =  [w, line_down];
pts_L1 = np.array([pt1,pt2], np.int32)
pts_L1 = pts_L1.reshape((-1,1,2))

#Background substractor function
fgbg = cv.createBackgroundSubtractorMOG2(detectShadows = True)

#Kernels for morphology
kernelOp = np.ones((3,3),np.uint8)
kernelOp2 = np.ones((5,5),np.uint8)
kernelCl = np.ones((11,11),np.uint8)

#Variables
persons = []
pid = 1           #Person ID
max_p_age = 5     
font = cv.FONT_HERSHEY_SIMPLEX

In [46]:
while(cap.isOpened()):
    ret, frame = cap.read()
    #Resize for any input video to have the same resolution/borders
    frame = cv.resize(frame,(640, 480))

    for i in persons:
        i.age_one()
        
    #Apply the background substractor to each frame
    fgmask = fgbg.apply(frame)
    
    #Binarization to delete shadows
    ret, imBin = cv.threshold(fgmask,200,255,cv.THRESH_BINARY)
    mask = cv.morphologyEx(imBin, cv.MORPH_OPEN, kernelOp)
    mask = cv.morphologyEx(mask , cv.MORPH_CLOSE, kernelCl)

    #Find the contours of the people
    _, contours, hierarchy = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    #Algorithm:
    #For every contour that is above 1024 size, we apply the cv.moments
    #To calculate the mass center/blob of the object (cx,cy) , and assing a rectangle over it
    #with the points of x,y,w,h, and assing it as a NEW object detected.
    #This is the "tracking" method.
    
    #After that,for every cobject detected that is around/over the down_limit
    #we check the rectangle x,y and the object/person attributes x,y coordinates
    #If both coordinates are between the height and width, then we consider that the persons was
    #already been tracked.
    
    #For counting, we just check if the object cross the line from the middle to the bottom,
    #then the person was going down, and set the person as done to free it from the proporties ID and stop tracking
    #else it was going upwards.
    #If the person/object stays in place for and age/frame of 5, then we take it out of the array, to stop tracking it.
    
    for cnt in contours:
        area = cv.contourArea(cnt)
        
        if area > areaTH:
            M = cv.moments(cnt)
            #Calculate x,y coordinate of center/blob
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            #Rectable boundaries of the person
            x,y,w,h = cv.boundingRect(cnt)
            new = True
            
            if cy in range(down_limit):
                for i in persons:
                    if abs(x-i.getX()) <= w and abs(y-i.getY()) <= h:
                        new = False
                        i.updateCoords(cx,cy)
                        if i.going_DOWN(line_down,line_down-1) == True:
                            counter += 1;
                        break
                    if i.getState() == '1':
                        if i.getDir() == 'down' and i.getY() > down_limit:
                            i.setDone()
                        else:
                            i.setDone()
                    if i.timedOut():
                        index = persons.index(i)
                        persons.pop(index)
                        del i
                if new == True:                                              #Detection of a new person on frame
                    p = Person.MyPerson(pid,cx,cy, max_p_age)                #Get the new ID,x,y
                    persons.append(p)                                        #Append it to the persons[] array
                    pid += 1                                                 #Increase the ID counter

            cv.circle(frame,(cx,cy), 5, (0,0,255), -1)                       #Draw blob/center circle
            img = cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)            #Draw rectable over the person                   
                    
    str_down = 'DOWN: '+ str(counter)
    frame = cv.polylines(frame,[pts_L1],False,line_down_color,thickness=2)   #Down line
    cv.putText(frame, str_down ,(10,90),font,0.5,(255,0,0),1,cv.LINE_AA)     #Draw down line

    cv.imshow('Frame',frame)
    
    #ESC for exit
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv.destroyAllWindows()